# 6.888 Final Project - Winograd Transforms

### Madeleine Waller & Diana Wofk

In this project, we use as our baseline this course's lab 4 WS 2D Architecture.  We build on this architecture in the following steps: <br />
- Include the baseline architecture with added energy computations for performance comparison.
- Redesign the PE array to perform element wise multiplications of the transformed weights/ifmaps (rather than convolution). Implement off chip pre/post processing of ifmaps, weights, and ofmaps.
- Move the post processing (reverse transform of ofmaps) onto the chip via a specialized adder array, similar to the PE array.
- Move the pre processing on chip (in addition to the post processing), using a similar specialized adder array to transform both the weights and ifmaps.

In [1]:
from nnsim.simulator import run_tb

%load_ext autoreload
%autoreload 2

## Baseline WS 2D Architecture

In [60]:
from ws_2d.tb import WSArchTB

ws_tb = WSArchTB()
run_tb(ws_tb, verbose=False, dump_stats=True)

ifmap glb depth: 16
psum glb depth: 32
weight glb depth: 0
ifmap:  [[[ 17   4   9  22]
  [ 18  -9   9  -1]
  [ -1   4   1  14]
  [  7   1   4   3]]

 [[ 14  -2   3  -8]
  [-25   6   8  -7]
  [ 22 -14   0  -1]
  [ 15  14   1   3]]

 [[ -8 -19  -3   1]
  [ 12  12  -3  -3]
  [-10 -14 -17  19]
  [ -5  -4 -12   7]]

 [[-16  -2  -8   3]
  [ -5 -11   0   4]
  [  0   3  -6  -3]
  [ -6  -3  -8 -17]]]
weights:  [[[[  1  -4 -16   4  -9   0   7   1]
   [ 11 -12   4  -6  -8  -5  -3   0]
   [-11   9   4 -15  14  18  11  -1]
   [-10  10  -4  12   2   9   3   7]]

  [[  0  17   1   4  18 -13 -12   9]
   [-11  19  -4  -7  19  14  18   9]
   [ -8  19  -2   8   9  -1   6   9]
   [  3 -10   2  13  -6  -1  -4  18]]

  [[  6   4  -7   5  -6   0  -6   6]
   [  5  -2   3 -10 -14   4   1   6]
   [ 23   9  -9  11 -13  -4   0  17]
   [ -7  -8   0  -6  11 -10 -11  -4]]]


 [[[ -4  19   9   0 -12   8 -10 -15]
   [ 11   3   9   3   8  -6 -10   6]
   [ -8  -6  -4   0  -3 -13  -6 -22]
   [  6 -16 -11   0  -7  15 -12 

## Winograd Transform - Pre/Post Processing Off Chip

In [64]:
from ws_2d_winograd_off.tb import WSArchTB

ws_tb = WSArchTB()
run_tb(ws_tb, verbose=False, dump_stats=True)

ifmap glb depth: 64
weight glb depth: 0
ifmaps:  [[[ -4  -3   5  -9]
  [ -4  -8 -10  -8]
  [  6   0  -8  -2]
  [-16  -7  21   0]]

 [[  7   0   3   6]
  [-11   0  15  -7]
  [ -5  -3   2   5]
  [ 12   4   0  12]]

 [[  7 -14  25  17]
  [ -1   3  13  -1]
  [  7  11 -10  -6]
  [  9   4   9  -1]]

 [[ -2 -19 -11   0]
  [  8   5  -1  -2]
  [  2   0  14   1]
  [-14 -12  15 -11]]]
weights:  [[[[  7  -2  -1  -1  12  -1   0 -15]
   [  2   5   2  13   0   2  16 -15]
   [ -4  14   9   6   8   3  16   3]
   [  2   1   1 -11   7  -1  18   0]]

  [[  4   2  -6   9  19   0  -3  -3]
   [-18  -1   2  10   9  -4  -2  11]
   [ -1  -5  13  10   0  -5 -20   5]
   [ -8 -19  15   6 -11 -12   8  -8]]

  [[ 12   6   5   4   1   8  -4   0]
   [  7   5 -11  -5   2  -4  17  -3]
   [ -1   7   3  -1  -7   3  10   4]
   [ -7  17 -14 -15   9   2  -5 -10]]]


 [[[ 23   1   5   3   4   5   7  -3]
   [ -2 -17  -7   2  10   5  -3 -12]
   [-27  11  -5  -4   1 -14  -3  20]
   [  5   3 -15  22 -14  19 -21  14]]

  [[ 16  -8

## Winograd Transform - Move Post Processing On Chip

In [63]:
from ws_2d_winograd_post_on.tb import WSArchTB

ws_tb = WSArchTB()
run_tb(ws_tb, verbose=False, dump_stats=True)

ifmap glb depth: 64
weight glb depth: 0
ifmaps:  [[[ -3  -5   1  -3]
  [-16  -8  -9  12]
  [  8   5  -5  -8]
  [ -5 -10  24 -22]]

 [[  5 -12  -1  -9]
  [-11 -11  17  -1]
  [ -7   5   0   7]
  [-18   3   7 -16]]

 [[  4  16   0   8]
  [  6  -7   0 -15]
  [ -4   2   7   0]
  [  7 -11  -1   0]]

 [[ 18  -2  -4   0]
  [ -9   1  -3 -17]
  [  0  -3 -16   0]
  [ 14   5   6 -13]]]
weights:  [[[[  4   1   5   4 -13   0  -7   0]
   [ -8  13  -8  -8   0 -10  -4  -8]
   [ 14   7   0  -3   9   4   4   0]
   [-16  -1  -5   6  10  -6   4  17]]

  [[ -6  16  -8   0   0   0  -8  -5]
   [ -6   3   3  22  13  -5   3   9]
   [  0   0   0   8  -8  -3   4   3]
   [  2  -3   9  21   4  -1   7  -5]]

  [[ -7   0 -25 -11  -3 -13 -10  -4]
   [-16  -4  -5   0  11   1   0   0]
   [  2 -10  -1   3 -13   8  10  -6]
   [ -2  -8   6 -10  -2  -8   0 -17]]]


 [[[ 13  -9   0  -2  -7  -8   9  -5]
   [  2  -8  -6   4   9  -1 -14  19]
   [ -8 -15  25  -4 -14  -6   3   1]
   [ -8  18  -5 -11   9  13  -8 -14]]

  [[ 10 -20

## Winograd Transform - Pre & Post Processing On Chip

In [65]:
from ws_2d_winograd_on_chip.tb import WSArchTB

ws_tb = WSArchTB()
run_tb(ws_tb, verbose=False, dump_stats=True)

ifmap glb depth: 64
weight glb depth: 0
ifmaps:  [[[  1  -9   8   6]
  [  5   7 -29   8]
  [ 18   4   1   6]
  [  3   6   0  -7]]

 [[-10  -9 -13 -10]
  [ 17   7  -6  -3]
  [  1  -6   0   7]
  [  0   3   8  -2]]

 [[  2  -3   0  -3]
  [ -5   1   5   1]
  [  4  -7 -12   4]
  [  6   0  13   9]]

 [[ 26  -2  -9  -7]
  [ -5   4 -21  13]
  [ -2  -2 -10  -1]
  [  0  -1   3  -5]]]
weights:  [[[[ -8   5   0   1  10 -10  -8  12]
   [-14 -13   8   2   1   0   0   0]
   [ 13  -3  -1  17   0  22  -1  13]
   [-16   1 -15   8 -12   2  -2 -11]]

  [[-16  -5  17  -2   9   0   8 -18]
   [ -4   9  -1   0  -4  11   2   0]
   [  6   5   2  15   0   2  -6   1]
   [ 15  -6 -11   9  -1   0   0   0]]

  [[  8  -8 -10   0 -16 -13  18   7]
   [  0  12 -10  -3   6   8   5 -25]
   [ -9   4  -3  25   9   5 -11   0]
   [  2  11   8  10  -9  14 -13  -5]]]


 [[[  4  -1 -10  -8 -15   5  12  -1]
   [ -4 -14   9  22 -17 -10  -6  -5]
   [  2  16  10   5  -7 -11  -7  15]
   [  0  -4  -1   8 -13  14   1  -2]]

  [[  4   2